In [13]:
from topmost.data import download_dataset

device = "cuda"  # or "cpu"
dataset_dir = "./datasets/20NG"

In [6]:
# download_dataset('Wikitext-103', cache_path='./datasets')

In [14]:
########################### Neural Topic Models ####################################
# dataset for neural topic models.
# For combinedTM, add contextual_embed=True.
import topmost

dataset = topmost.data.BasicDataset(dataset_dir, read_labels=True, device=device)

train_size:  11314
test_size:  7532
vocab_size:  5000
average length: 110.543


In [8]:
# model = topmost.models.ECRTM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
model = topmost.models.ETM(dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
model = model.to(device)


In [15]:
# create a trainer
# trainer = topmost.trainers.BasicTrainer(model, dataset, verbose=True)
trainer = topmost.trainers.BERTopicTrainer(dataset)

# train the model
top_words, train_theta = trainer.train()

In [16]:
 ########################### Evaluate ####################################
# 获取训练集和测试集的theta (doc-topic分布)
train_theta, test_theta = trainer.export_theta()

# evaluate topic diversity
TD = topmost.evaluations.compute_topic_diversity(top_words)
print(f"TD: {TD:.5f}")

TC = topmost.evaluations.compute_topic_coherence(dataset.train_texts, dataset.vocab, top_words)
print(f"TC: {TC:.5f}")
# evaluate clustering

results = topmost.evaluations.evaluate_clustering(test_theta, dataset.test_labels)
print(results)

# evaluate classification
results = topmost.evaluations.evaluate_classification(train_theta, test_theta, dataset.train_labels,
                                                      dataset.test_labels)
print(results)

TD: 0.66667
TC: 0.60239
{'Purity': 0.4115772703133298, 'NMI': 0.3687523669885181}
{'acc': 0.539166224110462, 'macro-F1': 0.533141206688229}


In [9]:
########################### test new documents ####################################
import torch
from topmost.preprocessing import Preprocessing

preprocessing = Preprocessing()

new_docs = [
    "This is a new document about space, including words like space, satellite, launch, orbit.",
    "This is a new document about Microsoft Windows, including words like windows, files, dos."
]

parsed_new_docs, new_bow = preprocessing.parse(new_docs, vocab=dataset.vocab)
new_theta = trainer.test(torch.as_tensor(new_bow, device=device).float())

print(new_theta.argmax(1))

parsing texts: 100%|██████████| 2/2 [00:00<?, ?it/s]


TypeError: expected string or bytes-like object